In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
import time

C:\Users\CHARU\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import utils
utils.__file__

'C:\\Users\\CHARU\\Desktop\\tflow\\assignment\\Assignment-1\\utils.py'

In [3]:
# Step 1: Read in data
mnist_folder = 'data/mnist'
utils.download_mnist(mnist_folder)
train, val, test = utils.read_mnist(mnist_folder, flatten=True)

data/mnist\train-images-idx3-ubyte.gz already exists
data/mnist\train-labels-idx1-ubyte.gz already exists
data/mnist\t10k-images-idx3-ubyte.gz already exists
data/mnist\t10k-labels-idx1-ubyte.gz already exists


In [4]:
# Define paramaters for the model
learning_rate = 0.001
batch_size = 128
n_epochs = 30
n_train = 60000
n_test = 10000

#Network parameters
n_hidden_1 = 500 # 1st layer number of features
n_hidden_2 = 500 # 2nd layer number of features
n_input = 784 # Mnist data input
n_classes = 10 # the number of classes mnist


In [5]:
# Step 2: Create datasets and iterator
# create training Dataset and batch it
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

# create testing Dataset and batch it
test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.batch(batch_size)

In [6]:
# create one iterator and initialize it with different datasets
iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
img, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)# initializer for train_data
test_init = iterator.make_initializer(test_data)# initializer for train_data

In [7]:
def multilayer_perceptron(x, weights, biases):
    #Hidden layer with relu activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    #hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    # output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [8]:
# Step 3: create weights and bias
# store layers weights and bias
weights = {
'h1' : tf.Variable(tf.random_normal([n_input, n_hidden_1])),
'h2' : tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
'out' : tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
'b1' : tf.Variable(tf.random_normal([n_hidden_1])),
'b2' : tf.Variable(tf.random_normal([n_hidden_2])),
'out': tf.Variable(tf.random_normal([n_classes]))
}


In [9]:
# construct model
logits = multilayer_perceptron(img, weights, biases)
# define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = label))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
# Step 7: calculate accuracy with test set
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

writer = tf.summary.FileWriter('./graphs/logreg', tf.get_default_graph())

with tf.Session() as sess:
   
    start_time = time.time()
    sess.run(tf.global_variables_initializer())

    # train the model n_epochs times
    for i in range(n_epochs): 	
        sess.run(train_init)	# drawing samples from train_data
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    print('Total time: {0} seconds'.format(time.time() - start_time))

    # test the model
    sess.run(test_init)			# drawing samples from test_data
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass

    print('Accuracy {0}'.format(total_correct_preds/n_test))
writer.close()

Average loss epoch 0: 2.596369568968928
Average loss epoch 1: 0.72011043100163
Average loss epoch 2: 0.4478580252029175
Average loss epoch 3: 0.2982355218987132
Average loss epoch 4: 0.19810693979609845
Average loss epoch 5: 0.1342195885884034
Average loss epoch 6: 0.08943394506133573
Average loss epoch 7: 0.058171929971336626
Average loss epoch 8: 0.03756488896581496
Average loss epoch 9: 0.024275137101256745
Average loss epoch 10: 0.015342873716077138
Average loss epoch 11: 0.009755145873452081
Average loss epoch 12: 0.006392736508792569
Average loss epoch 13: 0.004376597869816388
Average loss epoch 14: 0.0033058021574952573
Average loss epoch 15: 0.002612120338829321
Average loss epoch 16: 0.0021800758851811203
Average loss epoch 17: 0.0017336335628791604
Average loss epoch 18: 0.0013975321047202966
Average loss epoch 19: 0.0011450344236394348
Average loss epoch 20: 0.0009438754610993444
Average loss epoch 21: 0.0007627433079273202
Average loss epoch 22: 0.0006201446031661553
Averag